<a href="https://colab.research.google.com/github/bdi2357/iShares_ETF_holdings_downloader/blob/main/ishares_etf_holdings_downloader_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf/1467271812596.ajax?fileType=csv&fileName=IVV_holdings&dataType=fund&asOfDate=20200630
#
#

In [4]:

import random
import pandas as pd
import argparse
import os
from io import StringIO, BytesIO
import gzip
import shutil
import logging
import json
import multiprocessing as mp



In [ ]:
"""
yyyymmdd = datetime.strptime(self.holdings_date, '%Y-%m-%d').strftime('%Y%m%d')
        request = dict(url=f'{iShares.ROOT}{self.product_url}/{iShares.AJAX_REQUEST_CODE}',
                       params={'fileType': 'csv',
                               'fileName': f'{self.ticker}_holdings',
                               'dataType': 'fund',
                               'asOfDate': yyyymmdd},
                       headers={'User-Agent': random.choice(iShares.USER_AGENT_LIST)})

        print("request: %s"%request)
"""

'\nyyyymmdd = datetime.strptime(self.holdings_date, \'%Y-%m-%d\').strftime(\'%Y%m%d\')\n        request = dict(url=f\'{iShares.ROOT}{self.product_url}/{iShares.AJAX_REQUEST_CODE}\',\n                       params={\'fileType\': \'csv\',\n                               \'fileName\': f\'{self.ticker}_holdings\',\n                               \'dataType\': \'fund\',\n                               \'asOfDate\': yyyymmdd},\n                       headers={\'User-Agent\': random.choice(iShares.USER_AGENT_LIST)})\n\n        print("request: %s"%request)\n'

In [5]:
etfs_ishares = pd.read_html("https://www.ishares.com/us/products/etf-investments#/?productView=etf&pageNumber=1&sortColumn=totalNetAssets&sortDirection=desc&dataView=keyFacts&showAll=true")

In [6]:
len(etfs_ishares)

2

In [7]:
etfs_ishares[0].shape

(393, 10)

In [8]:
etfs_list = etfs_ishares[0]

In [9]:
etfs_list.head()

,Ticker,Name,12m Trailing Yield (%),As of,YTD Return (%),As of.1,Incept. Date,Gross Expense Ratio (%),Net Expense Ratio (%),Net Assets (USD)
0,IGLB,iShares 10+ Year Investment Grade Corporate Bo...,4.18,"Aug 30, 2022",-22.24,"Aug 30, 2022","Dec 08, 2009",0.06,0.06,1385865695
1,ILTB,iShares Core 10+ Year USD Bond ETF,3.99,"Aug 30, 2022",-22.37,"Aug 30, 2022","Dec 08, 2009",0.06,0.06,304482155
2,QLTA,iShares Aaa - A Rated Corporate Bond ETF,2.41,"Aug 30, 2022",-13.18,"Aug 30, 2022","Feb 14, 2012",0.15,0.15,1011262904
3,DVYA,iShares Asia/Pacific Dividend ETF,7.30,"Jul 31, 2022",-6.16,"Jul 31, 2022","Feb 23, 2012",0.49,0.49,33611015
4,STIP,iShares 0-5 Year TIPS Bond ETF,6.10,"Aug 30, 2022",-1.14,"Aug 30, 2022","Dec 01, 2010",0.03,0.03,12673720770


In [10]:
import dateutil

In [11]:
dateutil.parser.parse(etfs_list.loc[0,"Incept. Date"])

datetime.datetime(2009, 12, 8, 0, 0)

In [12]:
dateutil.parser.parse(etfs_list.loc[0,"Incept. Date"]).year

2009

In [ ]:
dateutil.parser.parse(etfs_list.loc[0,"Incept. Date"]).month

12

In [ ]:
dateutil.parser.parse(etfs_list.loc[0,"Incept. Date"]).day

8

In [13]:
from dateutil.parser import parse as parse_dt
def parse_dm(x):
  if x>9:
    return str(x)
  else:
    return "0" + str(x)
def date2fmt(dt):
    prsd = parse_dt(dt)
    return str(prsd.year)+"-"+parse_dm(prsd.month)+"-"+parse_dm(prsd.day)
    

In [14]:
date2fmt(etfs_list.loc[0,"Incept. Date"])

'2009-12-08'

In [15]:
etfs_list["dt_parsed"] = etfs_list.apply(lambda r: date2fmt(r["Incept. Date"]),axis=1)

In [16]:
etfs_list = etfs_list.sort_values(by = "dt_parsed")

In [17]:
etfs_list[:50]

,Ticker,Name,12m Trailing Yield (%),As of,YTD Return (%),As of.1,Incept. Date,Gross Expense Ratio (%),Net Expense Ratio (%),Net Assets (USD),dt_parsed
84,EWO,iShares MSCI Austria ETF,4.72,"Jul 31, 2022",-30.27,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,62502890,1996-03-12
85,EWK,iShares MSCI Belgium ETF,5.73,"Jul 31, 2022",-23.59,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,17842180,1996-03-12
89,EWC,iShares MSCI Canada ETF,2.14,"Jul 31, 2022",-11.51,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,3662234597,1996-03-12
106,EWQ,iShares MSCI France ETF,6.22,"Jul 31, 2022",-22.17,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,790168558,1996-03-12
108,EWG,iShares MSCI Germany ETF,4.56,"Jul 31, 2022",-31.51,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,1224766451,1996-03-12
115,EWH,iShares MSCI Hong Kong ETF,2.79,"Jul 31, 2022",-10.62,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,728773082,1996-03-12
83,EWA,iShares MSCI Australia ETF,7.18,"Jul 31, 2022",-8.86,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,1646164740,1996-03-12
122,EWJ,iShares MSCI Japan ETF,2.56,"Jul 31, 2022",-19.03,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,9661545792,1996-03-12
126,EWM,iShares MSCI Malaysia ETF,5.55,"Jul 31, 2022",-8.97,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,234387178,1996-03-12
127,EWW,iShares MSCI Mexico ETF,3.02,"Jul 31, 2022",-11.29,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,806156637,1996-03-12


In [ ]:
import os
os.getcwd()

'/Users/itaybendan/tmp_ipynb'

In [ ]:
import dateutil

In [ ]:
from dateutil.parser import parse as parse_dt
def parse_utl(dt):
    prsd = parse_dt(dt)
    return str(prsd.year)+"-"+str(prsd.month) + "-" + str(prsd.day)

In [ ]:
etfs_list["dt_start"] = etfs_list.apply(lambda r: parse_utl(r["Incept. Date"]),axis=1)

In [ ]:
etfs_list = etfs_list.sort_values(by = "dt_start")

In [ ]:
etfs_list[:50]

,Ticker,Name,12m Trailing Yield (%),As of,YTD Return (%),As of.1,Incept. Date,Gross Expense Ratio (%),Net Expense Ratio (%),Net Assets (USD),dt_parsed,dt_start
126,EWM,iShares MSCI Malaysia ETF,5.55,"Jul 31, 2022",-8.11,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,236601980,1996-3-12,1996-3-12
127,EWW,iShares MSCI Mexico ETF,3.02,"Jul 31, 2022",-2.84,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,965438584,1996-3-12,1996-3-12
83,EWA,iShares MSCI Australia ETF,7.18,"Jul 31, 2022",-4.03,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,1844356499,1996-3-12,1996-3-12
84,EWO,iShares MSCI Austria ETF,4.72,"Jul 31, 2022",-24.28,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,67872657,1996-3-12,1996-3-12
89,EWC,iShares MSCI Canada ETF,2.14,"Jul 31, 2022",-4.92,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,3913169688,1996-3-12,1996-3-12
106,EWQ,iShares MSCI France ETF,6.22,"Jul 31, 2022",-14.64,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,750251870,1996-3-12,1996-3-12
108,EWG,iShares MSCI Germany ETF,4.56,"Jul 31, 2022",-24.71,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,1375255527,1996-3-12,1996-3-12
115,EWH,iShares MSCI Hong Kong ETF,2.79,"Jul 31, 2022",-7.58,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,753571257,1996-3-12,1996-3-12
85,EWK,iShares MSCI Belgium ETF,5.73,"Jul 31, 2022",-16.35,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,19533273,1996-3-12,1996-3-12
140,EWL,iShares MSCI Switzerland ETF,1.97,"Jul 31, 2022",-16.18,"Jul 31, 2022","Mar 12, 1996",0.50,0.50,1439738950,1996-3-12,1996-3-12


In [ ]:
import requests

In [ ]:
t1 = requests.get("https://www.ishares.com/us/products/etf-investments#/?productView=etf&pageNumber=1&sortColumn=totalNetAssets&sortDirection=desc&dataView=keyFacts&showAll=true")

In [ ]:
big_s = t1.text

In [ ]:
import re
re.findall("\".*?\"",re.findall("<a href=\".*?\">IVV",big_s)[0])[0][1:-1]

'/us/products/239726/ishares-core-sp-500-etf'

In [ ]:
def find_etf_url(ticker,big_s):
    return re.findall("\".*?\"",re.findall("<a href=\".*?\">%s"%ticker,big_s)[0])[0][1:-1]

In [ ]:
find_etf_url("IWM",big_s)

'/us/products/239710/ishares-russell-2000-etf'

In [ ]:
find_etf_url("EWM",big_s)

'/us/products/239669/ishares-msci-malaysia-etf'

In [ ]:
etfs_list["url_add"] = etfs_list.apply(lambda r: find_etf_url(r["Ticker"],big_s),axis=1)